In [1]:
from finviz.screener import Screener
import os
import datetime
import pandas as pd
import time
import yfinance as yf
from datetime import datetime

In [2]:
def fetch_stocks():
    # filters = ['ind_exchangetradedfund','sh_avgvol_o2000', 'sh_price_o20']
    filters = ['sh_avgvol_o2000', 'sh_price_o20']
    stock_list = Screener(filters=filters, table='Performance', order='price')
    
    return stock_list

In [3]:
def cal_VRRS(ticker, ref, rolling_length):
    ticker_sma = ticker.rolling(rolling_length).mean()
    ticker_change = ticker - ticker_sma
    
    ref_sma = ref.rolling(rolling_length).mean()
    ref_change = ref - ref_sma
    
    return (ticker_change/ticker_sma - ref_change/ref_sma) * 100

In [4]:
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max()
    lowl = low.rolling(lookback).min()
    wr = 100 * ((close - highh) / (highh - lowl))
    return wr

In [5]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        if len(x) > 1:
            return float(x.replace('B', '')) * 1000000000
        return 1000000000.0
    if 'T' in x:
        if len(x) > 1:
            return float(x.replace('T', '')) * 1000000000000
        return 1000000000000000.0

    return 0.0

In [6]:
ticker_dict = {}

In [7]:
stock_list = fetch_stocks()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.20it/s]


In [ ]:
start_date = "2017-01-01"
# end_date = "2022-05-28"

end_date = datetime.today().date()

spy = yf.Ticker("SPY")
spy = spy.history(start=start_date, end=end_date, interval='5d')['Close'].dropna()

for stock in stock_list:
    
    ticker = stock['Ticker']
    
    avg_volume = value_to_float(stock['Avg Volume'])
    
    if avg_volume > 5000000:

        stock = yf.Ticker(ticker)    

        stock_close = stock.history(start=start_date, end=end_date, interval='5d')['Close'].dropna()
        stock = stock.history(start=start_date, end=end_date, interval='5d').dropna()

        williams = get_wr(stock['High'], stock['Low'], stock['Close'], 2).iloc[-1]

        # Calculate the VRRS indicator:
        rolling_length = 21 

        vrrs = cal_VRRS(stock_close, spy, rolling_length)

        if williams <= -85 or williams >= -15: 

            ticker_dict[ticker] = vrrs.iloc[-1]

            print(ticker, vrrs.iloc[-1])

UNG 3.212346548038805
QID 15.586350488984452
SJNK 3.336352192584333
GDX -12.768410156816422
TQQQ -26.620036556942043
IAU -0.18148997778604375
USHY 2.8453304540350235
QLD -13.403055966271394
SVXY 5.0578288922176045
SQQQ 15.347290267589697
- XLC: No data found for this date range, symbol may be delisted
HYG 2.3225559275436702
VCSH 5.624789323023787
VCIT 2.823214188981675
XBI 10.220208520983453
JNK 1.6374432537031933
SHV 6.9127387713363175
LQD 2.0990371251471123


In [ ]:
sorted_x = sorted(ticker_dict.items(), key=lambda x: x[1])

In [ ]:
for x in sorted_x:
    if abs(x[1]) > 10: 
        print(x[0], (round(x[1], 2)))